https://github.com/fmfn/BayesianOptimization

In [1]:
#read dfs# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
train_X=pd.read_csv("../input/vol1outputs/train_x.csv")
train_y=pd.read_csv("../input/vol1outputs/train_y.csv")
normalized_test_df=pd.read_csv("../input/vol1outputs/normalized_test_df.csv")
df_test1=pd.read_csv("../input/vol1outputs/df_test1.csv")

In [2]:
train_X=train_X.drop(columns=train_X.columns[0])
train_y=train_y.drop(columns=train_y.columns[0])
normalized_test_df=normalized_test_df.drop(columns=normalized_test_df.columns[0])
df_test1=df_test1.drop(columns=df_test1.columns[0])

In [3]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import ShuffleSplit

def xgboostcv(max_depth,
              learning_rate,
              n_estimators,
              gamma,
              min_child_weight,
              max_delta_step,
              subsample,
              reg_lambda,
              reg_alpha,
              colsample_bytree,
              colsample_bylevel,
              colsample_bynode,
              verbosity=0,
              nthread=-1):
    
    model=xgb.XGBRegressor(max_depth=int(round(max_depth)),
                           learning_rate=learning_rate,
                           n_estimators=int(round(n_estimators)),
                           gamma=gamma,
                           min_child_weight=min_child_weight,
                           max_delta_step=int(round(max_delta_step)),
                           subsample=subsample,
                           reg_lambda=reg_lambda,
                           reg_alpha=reg_alpha,
                           colsample_bytree=colsample_bytree,
                           colsample_bylevel=colsample_bylevel,
                           colsample_bynode=colsample_bynode,
                           verbosity=verbosity,
                           nthread=nthread,
                           tree_method='gpu_hist')
    
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    result = cross_val_score(model,train_X,train_y,scoring="neg_mean_squared_error",cv=cv).mean()
    
    return result


# Load data set and target values
xgboostBO = BayesianOptimization(f = xgboostcv,
                                 pbounds = {'max_depth': (5, 10),
                                          'learning_rate': (0.01, 0.3),
                                          'n_estimators': (50, 1000),
                                          'gamma': (0, 0.99),
                                          'min_child_weight': (1, 10),
                                          'max_delta_step': (0, 10),
                                          'subsample': (0.5, 0.99),
                                          'reg_lambda': (1, 100),
                                          'reg_alpha': (1, 100),
                                          'colsample_bytree' :(0.5, 0.99),
                                          'colsample_bylevel' :(0.5, 0.99),
                                          'colsample_bynode' :(0.5, 0.99)
                                          },
                                random_state=1)

xgboostBO.maximize(init_points=5, n_iter=50)
print('-'*53)

print('Final Results')
for res in xgboostBO.res:
    print('XGBOOST: %f' % res['target'])

print('*'*53)
print('best params ==>', xgboostBO.max)

|   iter    |  target   | colsam... | colsam... | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -5.445e+0 |  0.7043   |  0.853    |  0.5001   |  0.2993   |  0.05256  |  0.9234   |  5.931    |  4.11     |  426.9    |  54.34    |  42.5     |  0.8358   |
|  2        | -5.135e+0 |  0.6002   |  0.9303   |  0.5134   |  0.6638   |  0.131    |  5.587    |  5.702    |  2.783    |  810.7    |  96.86    |  32.03    |  0.8392   |
|  3        | -5.18e+06 |  0.9294   |  0.9384   |  0.5417   |  0.03866  |  0.05925  |  8.781    |  5.492    |  4.79     |  960.0    |  53.78    |  69.5     |  0.6546   |
|  4        | -5.273e+0 |  0.8364   |  0.909    |  0.509    |  0.7426   |  0.2968   |  7.482    |  6.402    |  8.104    |  148.1    |  45.34    |  90.

In [4]:
param_dict = xgboostBO.max['params'] 
param_dict['tree_method']='gpu_hist'

# we need to round some float values to integer since these parameters works only in integer
param_dict['max_depth']=int(round(param_dict['max_depth']))
param_dict['n_estimators']=int(round(param_dict['n_estimators']))
param_dict['max_delta_step']=int(round(param_dict['max_delta_step']))


In [5]:
best_xgb_model = xgb.XGBRegressor()
best_xgb_model.set_params(**param_dict)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=0.870775219488481,
             colsample_bynode=0.5059601984870159,
             colsample_bytree=0.9659705635318063, gamma=0.8968843524222182,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.15898134999123564, max_delta_step=0, max_depth=6,
             min_child_weight=3.89657602973688, missing=nan,
             monotone_constraints=None, n_estimators=329, n_jobs=None,
             num_parallel_tree=None, random_state=None,
             reg_alpha=33.2939268552398, reg_lambda=15.830287605007618,
             scale_pos_weight=None, subsample=0.7788690674320253,
             tree_method='gpu_hist', validate_parameters=None, verbosity=None)

In [6]:
best_xgb_model.fit(train_X,train_y)

XGBRegressor(base_score=0.5, booster='gbtree',
             colsample_bylevel=0.870775219488481,
             colsample_bynode=0.5059601984870159,
             colsample_bytree=0.9659705635318063, gamma=0.8968843524222182,
             gpu_id=0, importance_type='gain', interaction_constraints='',
             learning_rate=0.15898134999123564, max_delta_step=0, max_depth=6,
             min_child_weight=3.89657602973688, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=329, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=33.2939268552398, reg_lambda=15.830287605007618,
             scale_pos_weight=1, subsample=0.7788690674320253,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [7]:
y_pred = best_xgb_model.predict(normalized_test_df)

In [8]:
# prediction results
pred_df=pd.DataFrame(zip(df_test1['sector_id'], y_pred), columns=['sector_id', 'predictions'])
submission_dff=pred_df.groupby('sector_id').sum()
ss=submission_dff.reset_index()

ss=ss.rename(columns={"sector_id": "Sektor_Id", "predictions": "Expected"})

In [9]:
ss.to_csv('out_bayesian.csv', index=False)